# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [2]:
# Импортируем все необходимые библиотеки 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from joblib import dump
from tqdm import tqdm

In [3]:
# Открытие файла /datasets/users_behavior и сохранение его в переменную df
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('users_behavior.csv')

# Вывод таблицы
display(df.sample)

# Вывод информации о типах данных в таблицах  
df.info()

df.describe()

<bound method NDFrame.sample of       calls  minutes  messages   mb_used  is_ultra
0      40.0   311.90      83.0  19915.42         0
1      85.0   516.75      56.0  22696.96         0
2      77.0   467.66      86.0  21060.45         0
3     106.0   745.53      81.0   8437.39         1
4      66.0   418.74       1.0  14502.75         0
...     ...      ...       ...       ...       ...
3209  122.0   910.98      20.0  35124.90         1
3210   25.0   190.36       0.0   3275.61         0
3211   97.0   634.44      70.0  13974.06         0
3212   64.0   462.32      90.0  31239.78         0
3213   80.0   566.09       6.0  29480.52         1

[3214 rows x 5 columns]>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [4]:
# вывод количества пропущенных значений
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [5]:
df.duplicated().sum()

0

**Вывод:** в таблицах отсутствуют пропущенные значения и дупликаты, дополниельной предобработки не требуется 

## Разбейте данные на выборки

Разделим исходные данные на три выборки: обучающую, валидационную и тестовую. Так как у нас нет спрятанной тестовой выборки, то данные нужно разбить на три части в соотношении 3:1:1

In [5]:
# Разделим исходные данные на тестовую и обучающую выборки
df_train, df_test = train_test_split(df, test_size=0.2, random_state=59429, stratify=df['is_ultra'])

# Теперь разделим тренировачные данные и отдадим 20 процентов валидационной выборке 
df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=59429, stratify=df_train['is_ultra'])

# Выведем полученные таблицы
display(df_train)
display(df_test)
display(df_valid)

,calls,minutes,messages,mb_used,is_ultra
341,108.0,728.68,4.0,14340.84,1
1803,54.0,386.47,68.0,27401.56,0
888,46.0,308.33,23.0,18865.35,0
1172,144.0,1031.79,99.0,40174.34,1
412,89.0,716.92,22.0,29139.66,1
...,...,...,...,...,...
2513,39.0,242.71,0.0,20480.11,0
3004,108.0,772.08,91.0,14948.40,0
439,52.0,342.10,4.0,17785.85,0
516,53.0,385.59,63.0,16987.84,0


,calls,minutes,messages,mb_used,is_ultra
1151,53.0,459.43,78.0,21131.38,0
1633,87.0,638.31,76.0,28239.49,1
827,54.0,448.87,0.0,9670.72,0
280,68.0,413.20,29.0,13526.61,1
2970,56.0,382.18,12.0,21071.42,0
...,...,...,...,...,...
346,54.0,394.52,59.0,16207.41,0
1097,69.0,449.93,64.0,15557.04,0
1477,98.0,703.16,4.0,25391.67,1
2836,56.0,433.63,22.0,20869.68,0


,calls,minutes,messages,mb_used,is_ultra
1409,40.0,247.63,0.0,21420.44,0
2967,80.0,583.37,58.0,20898.57,1
2105,102.0,760.64,0.0,23166.00,1
1675,65.0,426.35,75.0,12099.84,1
1929,34.0,239.76,62.0,10793.22,0
...,...,...,...,...,...
2431,55.0,347.15,15.0,10648.36,0
2950,97.0,705.23,30.0,14103.81,0
1859,184.0,1203.11,52.0,43931.39,1
1303,81.0,625.49,126.0,16538.89,1


In [6]:
# Создадим переменные для признаков и целевого признака тренеровочной выборки: 
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# Создадим переменные для признаков и целевого признака тестовой выборки: 
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

# Создадим переменные для признаков и целевого признака валидационной выборки: 
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# Выведем результаты
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(2056, 4)
(2056,)
(643, 4)
(643,)
(515, 4)
(515,)


**Вывод:** Весь датафрейм разбит на три выборки: тренеровачная(60%), тестовая(20%) и валидационная(20%). Также для этих выборок созданы переменные с признаками. 

## Исследуйте модели

Теперь определим качество моделей, меняя им гиперпараметры 

In [7]:
# Настроим гиперпараметры дерева решений от 1 до 5
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=9058, max_depth=depth)
    
    model.fit(features_train, target_train)
    
    predictions_valid = model.predict(features_valid)
    
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7533980582524272
max_depth = 2 : 0.7805825242718447
max_depth = 3 : 0.7980582524271844
max_depth = 4 : 0.7902912621359224
max_depth = 5 : 0.8019417475728156


**Вывод:** Модель не стала лучше, поэтому необходимо использовать другую модель 

In [8]:
%%time
# Обучим модель случайного леса, при котором будет выявлена лучшая метрика валидации 
best_model = None
best_result = 0
for est in tqdm(range(1, 15)):
    for depth in range(5, 8):
        model = RandomForestClassifier(random_state=9058, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

100%|██████████| 14/14 [00:01<00:00, 13.24it/s]

Accuracy наилучшей модели на валидационной выборке: 0.8135922330097087
CPU times: user 1.01 s, sys: 18.7 ms, total: 1.03 s
Wall time: 1.06 s


**Вывод:** В результате использования случайного леса с числом деревьев от 1 до 15 и глубинной от 5 до 8, выявлена лучшая метрика валидации   результат, которой составляет 0,81 

Теперь попробуем применить модель логистической регрессии и посмотреть на результат: 

In [9]:
model = LogisticRegression(random_state=9058, solver='lbfgs', max_iter=1000)

model.fit(features_train, target_train)

print("Accuracy:", model.score(features_valid, target_valid))


Accuracy: 0.7067961165048544


**Выводы:** Самое высокое качество показала модель случайного леса. На второе место стоит поставить логистическую регрессию, так как у неё меньше всего параметров, следовательно выше скорость работы и ей не грозит переобучение, в отличие от дерева решений 

## Проверьте модель на тестовой выборке

In [10]:
# Дообучим модель на тренеровачно-валидационной выборке 
features = pd.concat([features_train, features_valid])
target = pd.concat([target_train, target_valid])

# Проверка лучшей модели на тестовой выборке
best_model = best_model.fit(features, target)
pred_test = best_model.score(features_test, target_test)

print("Accuracy на тестовой выборке:", pred_test)

Accuracy на тестовой выборке: 0.7682737169517885


**Вывод:** В результате проверки лучшей модели на тестовой выборки значение Accuracy получилось: 0.77

## Проверка модели на адекватность

In [11]:
dumm = DummyClassifier(strategy='most_frequent', random_state=1)
dumm.fit(features_train, target_train)
dumm.predict(features_test)
dumm.score(features_test, target_test)

0.6936236391912908

**Вывод:** Accuracy при дисбалансе в таргете, составялет 69 %. Модель в целом можно расценивать, как адекватную.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [X] Ячейки с кодом расположены в порядке исполнения
- [X] Выполнено задание 1: данные загружены и изучены
- [X] Выполнено задание 2: данные разбиты на три выборки
- [X] Выполнено задание 3: проведено исследование моделей
    - [X] Рассмотрено больше одной модели
    - [X] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [X] Написаны выводы по результатам исследования
- [X] Выполнено задание 3: Проведено тестирование
- [X] Удалось достичь accuracy не меньше 0.75
